In [8]:
# %%
import pandas as pd
import numpy as np
from sklearn import linear_model
import requests
from nba_api.stats import endpoints
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from matplotlib import pyplot as plt
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playergamelog
import datetime
from retrying import retry
import requests.exceptions

In [10]:
# %%
# convert raw data to dataframe and convert date col 
NBA_CSV_PATH = './data/NBA_player_box_score_stats_1950_2022.csv'
nba_csv = pd.read_csv(NBA_CSV_PATH)
nba_csv['GAME_DATE'] = pd.to_datetime(nba_csv['GAME_DATE'])
nba_csv['GAME_DATE'].head()

/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_48660/1675125041.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nba_csv['GAME_DATE'] = pd.to_datetime(nba_csv['GAME_DATE'])


0   1946-11-19
1   1946-11-23
2   1946-11-22
3   1946-11-26
4   1946-11-29
Name: GAME_DATE, dtype: datetime64[ns]

In [17]:
nba_csv[nba_csv["PLUS"] == 2021]

,Unnamed: 0,Season,Game_ID,PLAYER_NAME,Team,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
1237066,433105,2021,22100474,Aaron Gordon,DEN,2021-12-22,DEN @ OKC,L,25,5,...,4.0,4.0,2.0,1.0,0.0,1.0,0.0,12,-16.0,1
1237067,433106,2021,22100434,Aaron Gordon,DEN,2021-12-17,DEN @ ATL,W,30,5,...,3.0,3.0,3.0,0.0,1.0,4.0,1.0,16,6.0,1
1237068,433107,2021,22100424,Aaron Gordon,DEN,2021-12-15,DEN vs. MIN,L,27,2,...,2.0,4.0,1.0,0.0,0.0,2.0,0.0,8,-28.0,1
1237069,433137,2021,22000448,Aaron Gordon,DEN,2021-05-11,DEN @ CHA,W,29,8,...,3.0,6.0,3.0,2.0,0.0,0.0,0.0,17,11.0,1
1237070,433138,2021,22001008,Aaron Gordon,DEN,2021-05-07,DEN @ UTA,L,30,5,...,5.0,6.0,1.0,1.0,0.0,1.0,1.0,10,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269950,1284050,2021,22000214,Zion Williamson,NOP,2021-01-19,NOP @ UTA,L,35,14,...,4.0,5.0,3.0,1.0,0.0,2.0,4.0,32,-8.0,1
1269951,1284049,2021,22000227,Zion Williamson,NOP,2021-01-21,NOP @ UTA,L,34,9,...,0.0,3.0,2.0,0.0,0.0,1.0,0.0,27,-21.0,1
1269952,1284048,2021,22000246,Zion Williamson,NOP,2021-01-23,NOP @ MIN,L,38,6,...,7.0,11.0,3.0,1.0,1.0,2.0,3.0,19,-4.0,1
1269953,1284047,2021,22000279,Zion Williamson,NOP,2021-01-27,NOP vs. WAS,W,33,12,...,1.0,2.0,2.0,2.0,1.0,2.0,3.0,32,4.0,1


In [12]:
# get the players that scored the most ft in a game in a given time range
# return value: dataframe
def get_most_ftm_in1game_in_range(start, end, df):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    range = pd.date_range(start, end)
    df = df[df['GAME_DATE'].isin(range)]
    max_ftm_in_game = df['FTM'].max()
    players_with_max = df.loc[df['FTM'] == max_ftm_in_game]
    
    return players_with_max['PLAYER_NAME'].unique()

# test from 2000-2022
get_most_ftm_in1game_in_range('2000','2022',nba_csv)


array(['Anthony Davis'], dtype=object)